## Sentiment Analysis using RNN

Text Classification Task: Sentiment Analysis

- Data Preparation: Today we use a dataset (IMDb) for movie reviews. Each review is labeled as either positive or negative.

- Preprocessing: Tokenize the text and convert words to integers. Pad sequences to ensure they have the same length.

- Model Definition:
using an RNN layer to capture the sequential nature of the reviews.
Add a Dense layer with a sigmoid activation for binary classification.

- Training:
Train the model on the training dataset.

- Evaluation:
Test on a separate validation set and evaluate performance using metrics like accuracy or F1-score.

**Problem Statement:**

In this, we have to predict the number of positive and negative reviews based on sentiments by using RNN archticture. This is workable example on Many to One type as it takes sentances and output if it's negative or positive.

In [1]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional

from numpy import asarray
from numpy import zeros

from numpy import asarray
from numpy import zeros
from tensorflow.keras.layers import GRU

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding
from tensorflow.keras.layers import Flatten


import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import unicodedata
import html
stop_words = stopwords.words('english')

In [2]:
raw_data = pd.read_csv("IMDB Dataset.csv")
print(raw_data.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [3]:
raw_data.sentiment.value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [4]:
X = raw_data['review']  # Features: reviews
raw_data['label'] = raw_data['sentiment'].map({'positive': 1, 'negative': 0})
y = raw_data['label']  # Labels: sentiment (positive/negative)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 40000
Testing set size: 10000


### Data Prerocessing Pipeline

- remove_special_chars(text)

Purpose: Clean the input text by removing special characters and HTML entities.

Steps:

Compile a regex to match multiple spaces.

Convert text to lowercase.

Replace specific HTML character codes with their corresponding characters.

Replace newline characters and HTML tags with appropriate representations.

Use html.unescape to convert any remaining HTML entities.

Replace multiple spaces with a single space.

- remove_non_ascii(text)

Purpose: Eliminate non-ASCII characters from the text.

Steps:

Normalize the text to a compatible Unicode format.

Encode to ASCII, ignoring non-ASCII characters.

Decode back to UTF-8 format.

- to_lowercase(text)

Purpose: Convert all characters in the text to lowercase.

Steps:

Simply return the text converted to lowercase.

- remove_punctuation(text)

Purpose: Strip punctuation from the text.

Steps:

Create a translation table that maps punctuation characters to None.

Use the translation table to translate the text.

- replace_numbers(text)

Purpose: Remove all integer occurrences from the text.

Steps:

Use a regex to find and replace all digits with an empty string.

- remove_whitespaces(text)

Purpose: Trim leading and trailing whitespace from the text.

Steps:

Return the text after applying the strip() method.

- remove_stopwords(words, stop_words)

Purpose: Filter out common stopwords from a list of words.

Steps:

Return a list of words that are not present in the provided stop_words set.

- stem_words(words)

Purpose: Apply stemming to a list of words.

Steps:

Create an instance of a stemmer.

Return a list of stemmed words using the stemmer.

- lemmatize_words(words)

Purpose: Lemmatize words in the text to their base form.

Steps:

Create an instance of a lemmatizer.

Return a list of lemmatized words.

- lemmatize_verbs(words)

Purpose: Specifically lemmatize verbs in the text.

Steps:

Create an instance of a lemmatizer.

Return a string of lemmatized verbs, maintaining space between words.

- text2words(text)

Purpose: Tokenize the input text into a list of words.

Steps:

Use a word tokenizer to split the text into individual words and return the list.

In [5]:

def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)


def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)


def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]


def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
  return word_tokenize(text)

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    # words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [6]:
def normalize_corpus(corpus):
  return [normalize_text(t) for t in corpus]

In [7]:
proc_X_train = normalize_corpus(X_train)
proc_X_test = normalize_corpus(X_test)

### Building Pre-trained word embeddings using Glove6B With Bi-Directional LSTM

Overview of Layers

- Embedding Layer:

This layer initializes with the pre-trained GloVe embeddings.

You will need to load the GloVe vectors and create an embedding matrix where each word index corresponds to its GloVe vector.

- Bidirectional LSTM Layer:

This layer processes sequences in both forward and backward directions, capturing context from both sides.
It consists of two LSTM layers: one for the forward pass and another for the backward pass.

- Dense Layer(s):

Typically, you'll have one or more fully connected layers to output your final predictions.
The last dense layer often uses a softmax activation for classification tasks.

- Output Layer:

This layer generates the final predictions, which can be class labels, probabilities, etc.

In [8]:
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file I added it above when I used stanford api)
embeddings_index = dict()
with open('glove.6B.300d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using LSTM
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))  # LSTM layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Bidirectional(LSTM(100)))  # Another LSTM layer
model.add(Dropout(0.5))  # Another Dropout layer
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=20, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))

Loaded 400000 word vectors.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          36892500  
                                                                 
 bidirectional (Bidirectiona  (None, 100, 200)         320800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 100, 200)          0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)              

In [9]:
# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file)
embeddings_index = dict()
with open('glove.6B.300d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using Vanilla RNN
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(SimpleRNN(100, return_sequences=False))  # Simple RNN layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=20, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))


Loaded 400000 word vectors.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 300)          36892500  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 36,932,701
Trainable params: 40,201
Non-trainable params: 36,892,500
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 43s 34ms/step - loss: 0.6666 - acc: 0.6019
Epoch 2/20
1250/1250 [====================

In [10]:
# Prepare tokenizer
t = Tokenizer()
t.fit_on_texts(proc_X_train)  # Fit on training data only
vocab_size = len(t.word_index) + 1

# Integer encode the training documents
encoded_train_docs = t.texts_to_sequences(proc_X_train)
# Integer encode the testing documents
encoded_test_docs = t.texts_to_sequences(proc_X_test)

# Pad documents to a max length of 100 words (adjust as necessary)
max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

# Load the whole embedding into memory (make sure to have the GloVe file)
embeddings_index = dict()
with open('glove.6B.300d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

# Create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define model using GRU
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(GRU(100))  # GRU layer
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(1, activation='sigmoid'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# Summarize the model
model.summary()

# Fit the model
model.fit(padded_train_docs, y_train, epochs=20, batch_size = 32, verbose=1)  # Reduced epochs for quicker training

# Evaluate the model on the test set
loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print('Test Accuracy: %f' % (accuracy * 100))


Loaded 400000 word vectors.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 300)          36892500  
                                                                 
 gru (GRU)                   (None, 100)               120600    
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 37,013,201
Trainable params: 120,701
Non-trainable params: 36,892,500
_________________________________________________________________
Epoch 1/20
1250/1250 [==============================] - 9s 7ms/step - loss: 0.4430 - acc: 0.7943
Epoch 2/20
1250/1250 [=====================

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, GRU
from tensorflow.keras.callbacks import EarlyStopping
from numpy import asarray, zeros

t = Tokenizer()
t.fit_on_texts(proc_X_train)  
vocab_size = len(t.word_index) + 1

encoded_train_docs = t.texts_to_sequences(proc_X_train)
encoded_test_docs = t.texts_to_sequences(proc_X_test)

max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

embeddings_index = dict()
with open('glove.6B.300d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True))) 
model.add(Dropout(0.6))  
model.add(Bidirectional(LSTM(100))) 
model.add(Dropout(0.6)) 
model.add(Dense(1, activation='sigmoid'))  

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(padded_train_docs, y_train, epochs=20, batch_size=32, validation_data=(padded_test_docs, y_test), verbose=1, callbacks=[early_stopping])

loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Loaded 400000 word vectors.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 300)          36892500  
                                                                 
 bidirectional_2 (Bidirectio  (None, 100, 200)         320800    
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 100, 200)          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)          

In [12]:
import pandas as pd
import numpy as np
import re
import string
import html
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from numpy import asarray, zeros
import unicodedata
from nltk.stem import WordNetLemmatizer

raw_data = pd.read_csv("IMDB Dataset.csv")
raw_data['label'] = raw_data['sentiment'].map({'positive': 1, 'negative': 0})
X = raw_data['review']
y = raw_data['label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

stop_words = stopwords.words('english')

def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))

def remove_non_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def replace_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_whitespaces(text):
    return text.strip()

def remove_stopwords(words, stop_words):
    return [word for word in words if word not in stop_words]

def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def text2words(text):
    return word_tokenize(text)

def normalize_text(text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    words = lemmatize_words(words)
    return ' '.join(words)

def normalize_corpus(corpus):
    return [normalize_text(t) for t in corpus]

proc_X_train = normalize_corpus(X_train)
proc_X_test = normalize_corpus(X_test)

t = Tokenizer()
t.fit_on_texts(proc_X_train)
vocab_size = len(t.word_index) + 1

encoded_train_docs = t.texts_to_sequences(proc_X_train)
encoded_test_docs = t.texts_to_sequences(proc_X_test)

max_length = 100
padded_train_docs = pad_sequences(encoded_train_docs, maxlen=max_length, padding='post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post')

embeddings_index = dict()
with open('glove.6B.300d.txt', mode='rt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True))) 
model.add(Dropout(0.5)) 
model.add(Bidirectional(LSTM(100))) 
model.add(Dropout(0.5)) 
model.add(Dense(1, activation='sigmoid')) 

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(padded_train_docs, y_train, epochs=20, batch_size=32, validation_data=(padded_test_docs, y_test), verbose=1, callbacks=[early_stopping])

loss, accuracy = model.evaluate(padded_test_docs, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

model.save('sentiment_analysis_model')


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 300)          39677400  
                                                                 
 bidirectional_4 (Bidirectio  (None, 100, 200)         320800    
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 100, 200)          0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 200)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                

INFO:tensorflow:Assets written to: sentiment_analysis_model\assets


INFO:tensorflow:Assets written to: sentiment_analysis_model\assets


In [14]:
import pickle

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(t, f)